In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

%matplotlib inline

In [2]:
inp_df = pd.read_csv(r'TimeStamp_clients.csv')

Preprocessing data - deleted unnecessary columns to reduce noise


In [3]:
inp_df = inp_df.drop(['Suffix'], axis = 1)
inp_df = inp_df.drop(['Zip'], axis = 1)
inp_df = inp_df.drop(['Birthdate'], axis = 1)
inp_df = inp_df.drop(['Gender'], axis = 1)
inp_df = inp_df.drop(['Race'], axis = 1)
inp_df = inp_df.drop(['Creatingagency'], axis = 1)
inp_df = inp_df.drop(['Visitdt'], axis = 1)
inp_df = inp_df.drop(['Visittm'], axis = 1)
inp_df["Fullname"] = inp_df["Firstname"] +' '+inp_df["Lastname"]
inp_df = inp_df.drop(['Lastname'], axis = 1)
inp_df = inp_df.drop(['Firstname'], axis = 1)
temp = inp_df['Scandatetime'].str.split(' ')
inp_df['Scandate'] = temp.str.get(0)
inp_df = inp_df.drop(['Scandatetime'], axis = 1)
inp_df = inp_df[inp_df['Entrydate'].notnull()]
inp_df['Entrydate'] = pd.to_datetime(inp_df['Entrydate'])
inp_df['Scandate'] = pd.to_datetime(inp_df['Scandate'])
inp_df['Difference'] = inp_df.groupby(['Fullname'])['Scandate'].apply(lambda x: x.diff())



In [8]:
inp_df.head(4)

,Entrydate,Fullname,Scandate,Difference
0,2017-12-20,Ivan Castro,2018-01-02,NaT
1,2017-10-23,Carlos Ramos,2018-01-02,NaT
2,2014-04-12,Richard Bennett,2018-01-02,NaT
3,2017-06-25,Francis Lewis,2018-01-02,NaT


In [9]:
inp_df[inp_df['Fullname'] == "Richard Bennett"]

,Entrydate,Fullname,Scandate,Difference
2,2014-04-12,Richard Bennett,2018-01-02,NaT
621,2014-04-12,Richard Bennett,2018-01-05,3 days
1451,2014-04-12,Richard Bennett,2018-01-10,5 days
4151,2014-04-12,Richard Bennett,2018-01-25,15 days
4881,2014-04-12,Richard Bennett,2018-01-30,5 days
5328,2014-04-12,Richard Bennett,2018-02-01,2 days
6702,2014-04-12,Richard Bennett,2018-02-08,7 days
7575,2014-04-12,Richard Bennett,2018-02-12,4 days
8128,2014-04-12,Richard Bennett,2018-02-15,3 days
8872,2014-04-12,Richard Bennett,2018-02-20,5 days
